In [ ]:
import itertools
import json
import matplotlib.pyplot as plt
import numpy as np
import pymc3 as pm
import re
import statsmodels.api as sm

file_name = 'data/encode.json'
batch_size = 4096

with open(file_name, 'r') as f:
    data = json.load(f)

data = data['benchmarks']
data = filter(lambda x: x['run_type'] == 'iteration', data)

idxs = {}
idx_ctr = 0

def extract(x):
    global idx_ctr
    size = x['name']
    size = re.findall(r'8192/(\d{1,2})', size)
    size = int(size[0])
    real_time = x['real_time'] * 1000 # / (size * batch_size)
        
    if size in idxs:
        idx = idxs[size]
    else:
        idx = idx_ctr
        idxs[size] = idx_ctr
        idx_ctr += 1
    
    res = {
        "size": size,
        "idx": idx,
        "real_time": real_time
    }
    return res

data = list(map(extract, data))
data_grouped = itertools.groupby(data, lambda x: x['size'])

def map_seq(size, xs):
    xs = map(lambda x: x['real_time'], xs)
    return (size, np.array(list(xs)))

data_grouped = list(itertools.starmap(map_seq, data_grouped))

In [ ]:
n_rows = (len(data_grouped) + 1) // 2
fig, axs = plt.subplots(n_rows, 2, figsize=(16, 8 * n_rows))

for (idx, group) in enumerate(data_grouped):
    row = idx // 2
    col = idx % 2
    ax = axs[row, col]
    scale, series = group
    ax.set_title(f'scale: {scale * batch_size}', fontsize=18)
    sm.qqplot(data=series, ax=ax, line='s')

if len(data_grouped) % 2 == 1:
    axs[-1, -1].axis('off')

In [ ]:
y_idx = np.array([x['idx'] for x in data])
mean_scale = np.array([x['size'] for x in data]) * batch_size
sd_scale = np.sqrt(mean_scale)
y_obs = np.array([x['real_time'] for x in data])

with pm.Model() as model:
    num_groups = len(data_grouped)
    # hierarchical parameters
    mu_0 = pm.Normal('mu_0', mu=1, sd=0.5)
    sd_0 = pm.HalfNormal('sd_0', sd=0.5)
    # population parameters
    mu = pm.Normal('mu', mu=mu_0, sd=sd_0, shape=num_groups)
    sd = pm.HalfNormal('sd', sd=0.3)
    mu_scaled = pm.Deterministic('mu_scaled', mu[y_idx] * mean_scale)
    sd_scaled = pm.Deterministic('sd_scaled', sd * sd_scale)
    
    nu_minus_1 = pm.Exponential('nu_minus_1', lam=1)
    nu = pm.Deterministic('nu', 1 + nu_minus_1) # ensures dof >= 1
    
    y_pred = pm.StudentT('y_pred', mu=mu_scaled, sd=sd_scaled, nu=nu, observed=y_obs)
    
    trace = pm.sample(7000, chains=6, cores=6)

chain = trace[2000:]

In [ ]:
fig, axs = plt.subplots(3, 3, figsize=(16, 18))

titles = ['mu_0'] + [f'mu_{x[0] * batch_size}' for x in data_grouped]

axs = pm.plot_posterior(
    chain, 
    var_names=['mu_0', 'mu', 'sd', 'sd_0'], 
    credible_interval=0.95,
    ax=axs,
    round_to=4,
    kind='hist'
)

# ax_idx = 0
# for row in axs:
#     for ax in row:
#         ax.set_title(titles[ax_idx], fontsize=18)
#         ax_idx += 1
        
# _ = fig.suptitle('Amortised Operation Time (μs) - Encode', fontsize=20)